In [32]:
from pathlib import Path
import json
from collections import Counter
from tqdm import tqdm
import requests
import re
from collections import Counter

In [2]:
import pandas as pd
import numpy as np

In [3]:
datadir = Path("../data/2025-02-01-data-citation-corpus-v3.0-json/")

In [4]:
files = list(datadir.glob("*.json"))
# data = []
# def get_row_from_dict(item, corpus_id):
#     row = {"corpus_id": corpus_id}
#     for k in item.keys():
#         if item[k]:
#             row[k] = str(type(item[k]))
#         else:
#             row[k] = "missing"
#     return row
    
# for fp in tqdm(files):
#     txt = fp.read_text()
#     records = json.loads(txt)
#     for item in records:
#         row = get_row_from_dict(item, item["id"])
#         data.append(row)
#         for k in item.keys():
#             if type(item[k]) == dict:
#                 if k not in nested_data:
#                     nested_data[k] = []
#                 nested_data[k].append(get_row_from_dict(item[k], item["id"]))

# df_keys = pd.DataFrame(data)
# dfs_nested = {k: pd.DataFrame(v) for k, v in nested_data.items()}
df = pd.concat(pd.read_json(fp) for fp in files).set_index("id", verify_integrity=True)

In [5]:
rep = df["repository"].apply(lambda x: x.get("title", None))

In [6]:
rep.value_counts()

repository
European Nucleotide Archive               2276286
Cambridge Crystallographic Data Centre     727301
The Protein Data Bank                      403412
dbSNP Reference SNP                        233867
Gene Expression Omnibus (GEO)              200819
                                           ...   
GMBA-EarthEnv                                   1
University of Washington Libraries              1
Çiçek, Serhat Sezai                             1
GigaScience Journal                             1
Wyoming Data Repository                         1
Name: count, Length: 1855, dtype: int64

In [7]:
df_geo = df[rep=="Gene Expression Omnibus (GEO)"]

In [8]:
acc = df_geo.iloc[0]["dataset"]
print(acc)
api_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={acc.lower()}&targ=self&view=brief&form=text"
r = requests.get(api_url)

GSE48350


In [9]:
print(r.text)

^SERIES = GSE48350
!Series_title = Alzheimer's Disease Dataset
!Series_geo_accession = GSE48350
!Series_status = Public on Apr 21 2014
!Series_submission_date = Jun 27 2013
!Series_last_update_date = May 03 2022
!Series_pubmed_id = 18832152
!Series_pubmed_id = 23273601
!Series_pubmed_id = 22824372
!Series_pubmed_id = 20838618
!Series_pubmed_id = 23999428
!Series_pubmed_id = 23206327
!Series_summary = This dataset contains microarray data from normal controls (aged 20-99 yrs) and Alzheimer's disease cases, from 4 brain regions: hippocampus, entorhinal cortex, superior frontal cortex, post-central gyrus.   Changes in expression of synaptic and immune related genes were analyzed, investigating age-related changes and AD-related changes, and  region-specific patterns of change.
!Series_summary = These AD cases were processed simultaneously with the control cases (young and aged) included in GSE11882 (GSE11882 dataset contains data exclusively from normal control brains).
!Series_overall_de

In [10]:
acc_prefix = df_geo["dataset"].apply(lambda x: x[:3])
acc_prefix.value_counts()

dataset
GSE    177005
GSM     14453
GPL      6946
GDS      2415
Name: count, dtype: int64

In [11]:
df_geo_series = df_geo[acc_prefix=="GSE"]

In [12]:
print(len(df_geo_series))
print(df_geo_series["dataset"].nunique())

177005
76056


In [17]:
df_geo_series_sample = df_geo_series.drop_duplicates(subset="dataset").sample(n=500, random_state=12)
api_responses = []
for corpus_id, row in tqdm(df_geo_series_sample.iterrows(), total=len(df_geo_series_sample)):
    acc = row["dataset"]
    api_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={acc.lower()}&targ=self&view=brief&form=text"
    r = requests.get(api_url)
    api_responses.append({
        "id": corpus_id,
        "api_response": r.text,
    })

100%|██████████| 500/500 [04:10<00:00,  1.99it/s]


In [42]:
p = re.compile(r"!(.*?) = (.*)")
responses_fields = []
for item in api_responses:
    fields = Counter(m[0] for m in p.findall(item["api_response"]))
    row = {"id": item["id"]}
    for k, v in fields.items():
        row[k] = v
    responses_fields.append(row)

In [43]:
pd.DataFrame(responses_fields)

,id,Series_title,Series_geo_accession,Series_status,Series_submission_date,Series_last_update_date,Series_pubmed_id,Series_summary,Series_overall_design,Series_type,...,Series_sample_organism,Series_sample_taxid,Series_relation,Series_contact_department,Series_contact_laboratory,Series_web_link,Series_contact_fax,Series_contact_web_link,Series_gp_id,Series_citation
0,aa2dc50c-54e3-4a93-8a7a-e77aed49c232,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bc369301-3a7a-44f7-b961-cb1dfc1a54e5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7a59db47-d4b5-486b-9371-5ff910cb1277,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39db690e-0879-4afe-83d3-fe18ec4f4e5f,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,...,2.0,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,74c07f9a-7227-4261-8516-c36d4500a3b3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,c5cbdb0a-1eb5-4fb8-910b-36e1b390a933,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
496,dad33f83-4b1f-4a2f-9904-74e3b5d464f5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
497,29147cdf-1880-46ac-bbfe-987040b2f651,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
498,988c1df4-18a2-4da7-ab7c-c1ad4f4317fe,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_responses_fields = pd.DataFrame(responses_fields)

In [45]:
df_responses_fields["Series_contact_institute"].value_counts(dropna=False)

Series_contact_institute
1.0    498
NaN      2
Name: count, dtype: int64

In [39]:
fields.items()

dict_items([('Series_title', 1), ('Series_geo_accession', 1), ('Series_status', 1), ('Series_submission_date', 1), ('Series_last_update_date', 1), ('Series_pubmed_id', 1), ('Series_summary', 1), ('Series_overall_design', 1), ('Series_type', 1), ('Series_contributor', 2), ('Series_sample_id', 6), ('Series_contact_name', 1), ('Series_contact_email', 1), ('Series_contact_institute', 1), ('Series_contact_address', 1), ('Series_contact_city', 1), ('Series_contact_zip/postal_code', 1), ('Series_contact_country', 1), ('Series_supplementary_file', 1), ('Series_platform_id', 1), ('Series_platform_organism', 1), ('Series_platform_taxid', 1), ('Series_sample_organism', 1), ('Series_sample_taxid', 1), ('Series_relation', 2)])

In [23]:
item["api_response"]

'^SERIES = GSE133296\r\n!Series_title = Matched primary ovarian cancer, omental metastases, and non-omental metastases from 10 patients with high-grade serous ovarian carcinoma \r\n!Series_geo_accession = GSE133296\r\n!Series_status = Public on Jul 02 2020\r\n!Series_submission_date = Jun 25 2019\r\n!Series_last_update_date = Oct 01 2020\r\n!Series_pubmed_id = 32766252\r\n!Series_summary = Matched high-grade serous ovarian carcinoma samples collected from the ovary (ov), omental metastasis (om-met), and non-omental intraperitoneal metastasis (met) from 10 patients at the time of primary debulking surgery were analyzed for RNA expression by RNA sequencing.\r\n!Series_overall_design = FFPE blocks of matched high-grade serous ovarian carcinoma samples collected from the ovary (ov), omental metastasis (om-met), and non-omental intraperitoneal metastasis (met) from 10 patients were sectioned onto uncharged glass slides. One 4 Âµm H&E-stained section was used by a pathologist to circle the t